In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip "/content/gdrive/My Drive/Colab Notebooks/small.zip"

In [1]:
train_dir='small/train/'
validation_dir='small/validation/'
test_dir='small/test/'

In [2]:
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Reshape,AveragePooling2D, SeparableConv2D,Conv2D, MaxPooling2D
from keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import os

Using TensorFlow backend.


In [3]:
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Reshape,AveragePooling2D, SeparableConv2D,Conv2D, MaxPooling2D
from keras import backend as K

import matplotlib.pyplot as plt
import numpy as np

In [295]:
from keras.applications import mobilenet,mobilenetv2
net_model=mobilenet
net_description='mobilenet'

train_datagen = ImageDataGenerator(preprocessing_function=net_model.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=net_model.preprocess_input)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        target_size=(INPUT_SHAPE, INPUT_SHAPE),
        batch_size=BATCH_SIZE,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
train_generator.shuffle = False

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(INPUT_SHAPE, INPUT_SHAPE),
        batch_size=BATCH_SIZE,
        class_mode='binary')
validation_generator.shuffle = False


test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(INPUT_SHAPE, INPUT_SHAPE),
        batch_size=BATCH_SIZE,
        class_mode='binary')
test_generator.shuffle = False

Found 512 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [296]:
model = mobilenet.MobileNet(weights='imagenet',input_shape=(INPUT_SHAPE,INPUT_SHAPE, 3),include_top=False, pooling='avg')
train_features = model.predict_generator(train_generator)
validation_features = model.predict_generator(validation_generator)
test_features = model.predict_generator(test_generator)

In [297]:
print(train_features.shape, validation_features.shape, test_features.shape)

(512, 1024) (256, 1024) (256, 1024)


In [298]:
y_train = train_generator.labels[train_generator.index_array]
y_validation = validation_generator.labels[validation_generator.index_array]
y_test = test_generator.labels[test_generator.index_array]

In [299]:
print(len(y_train), len(y_validation), len(y_test))

512 256 256


## KNeighborsClassifier

In [301]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

classifier_1 = KNeighborsClassifier(n_neighbors=1)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.97      0.98      0.98       128
          1       0.98      0.97      0.98       128

avg / total       0.98      0.98      0.98       256



In [302]:
classifier_1 = KNeighborsClassifier(n_neighbors=30)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.97      0.99      0.98       128
          1       0.99      0.97      0.98       128

avg / total       0.98      0.98      0.98       256



In [303]:
classifier_1 = KNeighborsClassifier(n_neighbors=50)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.98      0.99      0.98       128
          1       0.99      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [304]:
classifier_1 = KNeighborsClassifier(n_neighbors=80)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.96      0.99      0.98       128
          1       0.99      0.96      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [305]:
classifier_1 = KNeighborsClassifier(n_neighbors=1)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(test_features)

print(classification_report(y_test, preds))

             precision    recall  f1-score   support

          0       0.98      0.98      0.98       128
          1       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



## SVC 

In [306]:
from sklearn.svm import SVC

classifier_2 = SVC(kernel='linear')
classifier_2.fit(train_features, y_train) 

preds = classifier_2.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.99      0.98       128
       dogs       0.99      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [307]:
from sklearn.svm import SVC

classifier_2 = SVC(kernel='poly')
classifier_2.fit(train_features, y_train) 

preds = classifier_2.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [308]:
from sklearn.svm import SVC

classifier_2 = SVC(kernel='linear')
classifier_2.fit(train_features, y_train) 

preds = classifier_2.predict(test_features)
print(classification_report(y_test, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.99      0.99       128
       dogs       0.99      0.98      0.99       128

avg / total       0.99      0.99      0.99       256



## RandomForest 

In [309]:
from sklearn import ensemble

classifier_3 = ensemble.RandomForestClassifier(n_estimators = 25)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [310]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 10)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.95      0.97      0.96       128
       dogs       0.97      0.95      0.96       128

avg / total       0.96      0.96      0.96       256



In [311]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 35)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       1.00      0.97      0.98       128
       dogs       0.97      1.00      0.98       128

avg / total       0.98      0.98      0.98       256



In [312]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 55)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [313]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 25)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(test_features)
print(classification_report(y_test, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.97      0.97       128
       dogs       0.97      0.98      0.97       128

avg / total       0.97      0.97      0.97       256



## ExtraTreesClassifier

In [314]:
classifier_4 = ensemble.ExtraTreesClassifier(n_estimators = 25)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.99      0.97      0.98       128
       dogs       0.97      0.99      0.98       128

avg / total       0.98      0.98      0.98       256



In [315]:
classifier_4 = ensemble.ExtraTreesClassifier(n_estimators = 10)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.95      0.98      0.96       128
       dogs       0.98      0.95      0.96       128

avg / total       0.96      0.96      0.96       256



In [316]:
classifier_4 = ensemble.ExtraTreesClassifier(n_estimators = 35)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.96      0.98      0.97       128
       dogs       0.98      0.96      0.97       128

avg / total       0.97      0.97      0.97       256



In [317]:
classifier_4 = ensemble.RandomForestClassifier(n_estimators = 55)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.99      0.98      0.98       128
       dogs       0.98      0.99      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [318]:
classifier_4 = ensemble.RandomForestClassifier(n_estimators = 35)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(test_features)
print(classification_report(y_test, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       1.00      0.98      0.99       128
       dogs       0.98      1.00      0.99       128

avg / total       0.99      0.99      0.99       256



In [321]:
from keras.applications import resnet
net_model=resnet

train_datagen = ImageDataGenerator(preprocessing_function=net_model.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=net_model.preprocess_input)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        target_size=(INPUT_SHAPE, INPUT_SHAPE),
        batch_size=BATCH_SIZE,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
train_generator.shuffle = False

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(INPUT_SHAPE, INPUT_SHAPE),
        batch_size=BATCH_SIZE,
        class_mode='binary')
validation_generator.shuffle = False


test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(INPUT_SHAPE, INPUT_SHAPE),
        batch_size=BATCH_SIZE,
        class_mode='binary')
test_generator.shuffle = False

Found 512 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [322]:
model = resnet.ResNet101(weights='imagenet',input_shape=(INPUT_SHAPE,INPUT_SHAPE, 3),include_top=False, pooling='avg')
train_features = model.predict_generator(train_generator)
validation_features = model.predict_generator(validation_generator)
test_features = model.predict_generator(test_generator)

171450368/171446536 [==============================] - 146s 1us/step


In [323]:
print(train_features.shape, validation_features.shape, test_features.shape)

(512, 2048) (256, 2048) (256, 2048)


In [324]:
y_train = train_generator.labels[train_generator.index_array]
y_validation = validation_generator.labels[validation_generator.index_array]
y_test = test_generator.labels[test_generator.index_array]

In [325]:
print(len(y_train), len(y_validation), len(y_test))

512 256 256


## KNeighborsClassifier

In [326]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

classifier_1 = KNeighborsClassifier(n_neighbors=1)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.98      0.98      0.98       128
          1       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [327]:
classifier_1 = KNeighborsClassifier(n_neighbors=30)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98       128
          1       1.00      0.97      0.98       128

avg / total       0.98      0.98      0.98       256



In [328]:
classifier_1 = KNeighborsClassifier(n_neighbors=50)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99       128
          1       1.00      0.98      0.99       128

avg / total       0.99      0.99      0.99       256



In [329]:
classifier_1 = KNeighborsClassifier(n_neighbors=80)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(validation_features)

print(classification_report(y_validation, preds))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99       128
          1       1.00      0.98      0.99       128

avg / total       0.99      0.99      0.99       256



#### Test best model:

In [344]:
classifier_1 = KNeighborsClassifier(n_neighbors=90)
classifier_1.fit(train_features, y_train) 

preds = classifier_1.predict(test_features)

print(classification_report(y_test, preds))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99       128
          1       1.00      0.98      0.99       128

avg / total       0.99      0.99      0.99       256



## SVC 

In [331]:
from sklearn.svm import SVC

classifier_2 = SVC(kernel='linear')
classifier_2.fit(train_features, y_train) 

preds = classifier_2.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      1.00      0.99       128
       dogs       1.00      0.98      0.99       128

avg / total       0.99      0.99      0.99       256



In [332]:
from sklearn.svm import SVC

classifier_2 = SVC(kernel='poly')
classifier_2.fit(train_features, y_train) 

preds = classifier_2.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.97      0.98      0.98       128
       dogs       0.98      0.97      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [333]:
from sklearn.svm import SVC

classifier_2 = SVC(kernel='linear')
classifier_2.fit(train_features, y_train) 

preds = classifier_2.predict(test_features)
print(classification_report(y_test, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.99      1.00      1.00       128
       dogs       1.00      0.99      1.00       128

avg / total       1.00      1.00      1.00       256



## RandomForest 

In [334]:
from sklearn import ensemble

classifier_3 = ensemble.RandomForestClassifier(n_estimators = 25)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [335]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 10)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.96      0.98      0.97       128
       dogs       0.98      0.96      0.97       128

avg / total       0.97      0.97      0.97       256



In [336]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 35)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.99      0.98       128
       dogs       0.99      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [337]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 55)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [338]:
classifier_3 = ensemble.RandomForestClassifier(n_estimators = 25)
classifier_3.fit(train_features, y_train) 

preds = classifier_3.predict(test_features)
print(classification_report(y_test, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.99      1.00      1.00       128
       dogs       1.00      0.99      1.00       128

avg / total       1.00      1.00      1.00       256



## ExtraTreesClassifier

In [339]:
classifier_4 = ensemble.ExtraTreesClassifier(n_estimators = 25)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [340]:
classifier_4 = ensemble.ExtraTreesClassifier(n_estimators = 10)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.95      0.97      0.96       128
       dogs       0.97      0.95      0.96       128

avg / total       0.96      0.96      0.96       256



In [341]:
classifier_4 = ensemble.ExtraTreesClassifier(n_estimators = 35)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.97      0.98       128
       dogs       0.97      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



In [342]:
classifier_4 = ensemble.RandomForestClassifier(n_estimators = 55)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(validation_features)
print(classification_report(y_validation, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.98      0.98      0.98       128
       dogs       0.98      0.98      0.98       128

avg / total       0.98      0.98      0.98       256



#### Test best model:

In [343]:
classifier_4 = ensemble.RandomForestClassifier(n_estimators = 35)
classifier_4.fit(train_features, y_train) 

preds = classifier_4.predict(test_features)
print(classification_report(y_test, preds, target_names=['cats', 'dogs']))

             precision    recall  f1-score   support

       cats       0.99      0.98      0.99       128
       dogs       0.98      0.99      0.99       128

avg / total       0.99      0.99      0.99       256

